In [1]:
import sys
import os
sys.path.append(os.path.abspath("/workspace/muellot_jetbot/jetbot"))

import time

# Basic Motion

## Importing Robot2 class

Execute the following block of code by selecting it and clicking ctrl + enter to create an NvidiaRacecar class.

In [2]:
from robot2 import Robot2

car = Robot2()

The Robot2 implements the Motors class, so it has two attributes throttle and steering.

We can assign values in the range [-1, 1] to these attributes. Execute the following to set the steering to 0.4.

In [3]:
car.steering = 0.3

The NvidiaRacecar class has two values steering_gain and steering_bias that can be used to calibrate the steering.

We can view the default values by executing the cells below.

In [4]:
print(car.steering_gain)

1.0


In [5]:
print(car.steering_offset)

0.09


The final steering value is computed using the equation

y = a * x + b

Where,

a is car.steering_gain
b is car.steering_offset
x is car.steering
y is the value written to the motor driver

You can adjust these values calibrate the car so that setting a value of 0 moves forward, and setting a value of 1 goes fully right, and -1 fully left.

To set the throttle of the car to 0.2, you can call the following.

Give JetRacer lots of space to move, and be ready on the manual override, JetRacer is fast

In [6]:
car.throttle = 0.0

The throttle also has a gain value that could be used to control the speed response. The throttle output is computed as
y = a * x
Where,

a is car.throttle_gain
x is car.throttle
y is the value written to the speed controller
Execute the following to print the default gain

In [7]:
print(car.throttle_gain)

1.0


Set the following to limit the throttle to half

In [8]:
car.throttle_gain = 1.0
car.steering_gain = 1.0
car.steering_offset = 0.09

Please note the throttle is directly mapped to the RC car. When the car is stopped and a negative throttle is set, it will reverse. If the car is moving forward and a negative throttle is set, it will brake.

## Link motors to traitlets
A really cool feature about these traitlets is that we can also link them to other traitlets! This is super handy because Jupyter Notebooks allow us to make graphical widgets that use traitlets under the hood. This means we can attach our motors to widgets to control them from the browser, or just visualize the value.

To show how to do this, let's create and display two sliders that we'll use to control our motors.

In [1]:
import ipywidgets.widgets as widgets
from IPython.display import display

# create two sliders with range [-1.0, 1.0]
throttle_slider = widgets.FloatSlider(description='left', min=-1.0, max=1.0, step=0.01, orientation='vertical')
steering_slider = widgets.FloatSlider(description='right', min=-1.0, max=1.0, step=0.01, orientation='horizontal')

# create a horizontal box container to place the sliders next to each other
slider_container = widgets.HBox([throttle_slider, steering_slider])

# display the container in this cell's output
display(slider_container)

You should see two vertical sliders displayed above.

HELPFUL TIP: In Jupyter Lab, you can actually "pop" the output of cells into entirely separate window! It will still be connected to the notebook, but displayed separately. This is helpful if we want to pin the output of code we executed elsewhere. To do this, right click the output of the cell and select Create New View for Output. You can then drag the new window to a location you find pleasing.

Try clicking and dragging the sliders up and down. Notice nothing happens when we move the sliders currently. That's because we haven't connected them to motors yet! We'll do that by using the link function from the traitlets package.

In [10]:
import traitlets

throttle_link = traitlets.link((throttle_slider, 'value'), (car, 'throttle'))
steering_link = traitlets.link((steering_slider, 'value'), (car, 'steering'))

In [13]:
car.stop()

## Attach functions to events
Another way to use traitlets, is by attaching functions (like forward) to events. These functions will get called whenever a change to the object occurs, and will be passed some information about that change like the old value and the new value.

Let's create and display some buttons that we'll use to control the robot.

In [2]:
# create buttons
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)

# display buttons
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([forward_button, middle_box, backward_button])
display(controls_box)

You should see a set of robot controls displayed above! But right now they wont do anything. To do that we'll need to create some functions that we'll attach to the button's on_click event.

In [ ]:
def stop(change):
    car.stop()

def step_forward(change):
    car.throttle = 0.5
    car.steering = 0.0
    time.sleep(0.5)
    car.stop()

def step_backward(change):
    car.throttle = -0.5
    car.steering = 0.0
    time.sleep(0.5)
    car.stop()

def step_left(change):
    car.steering = -1.0
    time.sleep = 0.5
    car.stop()

def step_right(change):
    car.steering = 1.0
    time.sleep = 0.5
    car.stop()

Now that we've defined the functions, let's attach them to the on-click events of each button

In [ ]:
# link buttons to actions
stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)

Now when you click each button, you should see the robot move!

## Heartbeat Killswitch
Here we show how to connect a 'heartbeat' to stop the robot from moving. This is a simple way to detect if the robot connection is alive. You can lower the slider below to reduce the period (in seconds) of the heartbeat. If a round-trip communication between browser cannot be made within two heartbeats, the 'status' attribute of the heartbeat will be set dead. As soon as the connection is restored, the status attribute will return to alive.

In [ ]:
from heartbeat import Heartbeat

heartbeat = Heartbeat()

# this function will be called when heartbeat 'alive' status changes
def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        car.stop()

heartbeat.observe(handle_heartbeat_status, names='status')

period_slider = widgets.FloatSlider(description='period', min=0.001, max=0.5, step=0.01, value=0.5)
traitlets.dlink((period_slider, 'value'), (heartbeat, 'period'))

display(period_slider, heartbeat.pulseout)

Try executing the code below to start the motors, and then lower the slider to see what happens. You can also try disconnecting your robot or PC.

In [ ]:
car.throttle = 0.5
car.steering = 1
# now lower the `period` slider above until the network heartbeat can't be satisfied

## Conclusion
That's it for this example notebook! Hopefully you feel confident that you can program your robot to move around now :)